In [ ]:
import torch
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [5]:
df.shape

(569, 31)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [7]:
X = df.drop(columns=['diagnosis'])
y  = df['diagnosis']

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
y_train

68     B
181    M
63     B
248    B
60     B
      ..
71     B
106    B
270    B
435    M
102    B
Name: diagnosis, Length: 455, dtype: object

In [11]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

Convert to torch tensors

In [12]:
X_train_tensor = torch.from_numpy(X_train_scaled)
X_test_tensor = torch.from_numpy(X_test_scaled)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [13]:
X_train_tensor.shape,X_test_tensor.shape

(torch.Size([455, 30]), torch.Size([114, 30]))

In [ ]:
class NN:

  def __init__(self,X):
      self.weights = torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
      self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)

  def forward(self,X):
    z = torch.matmul(X,self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self,y_pred,y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss

In [15]:
learning_rate = 0.1
epochs = 25

In [16]:
# Create the model
model = NN(X_train_tensor)

# Define loop
for epoch in range(epochs):

  # Forward Propagation
  y_pred = model.forward(X_train_tensor)

  # Calculate Loss
  loss =  model.loss_function(y_pred,y_train_tensor)

  # Backward Pass
  loss.backward()

  # Parameter Updates
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  model.weights.grad.zero_()
  model.bias.grad.zero_()

  print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")


Epoch: 1, Loss: 3.849841812502024
Epoch: 2, Loss: 3.7300000734494314
Epoch: 3, Loss: 3.610008823488334
Epoch: 4, Loss: 3.486624804822719
Epoch: 5, Loss: 3.361527203397543
Epoch: 6, Loss: 3.2354671858078645
Epoch: 7, Loss: 3.1090158436139514
Epoch: 8, Loss: 2.978962762561216
Epoch: 9, Loss: 2.8392600917854636
Epoch: 10, Loss: 2.6974011574210524
Epoch: 11, Loss: 2.5507564031884207
Epoch: 12, Loss: 2.4011486187319155
Epoch: 13, Loss: 2.253895442102762
Epoch: 14, Loss: 2.1088146667737213
Epoch: 15, Loss: 1.9680533042311985
Epoch: 16, Loss: 1.8332955139754938
Epoch: 17, Loss: 1.7048680265876377
Epoch: 18, Loss: 1.5847359714337084
Epoch: 19, Loss: 1.472320031461197
Epoch: 20, Loss: 1.367189724691891
Epoch: 21, Loss: 1.2732419858925776
Epoch: 22, Loss: 1.1931032091594342
Epoch: 23, Loss: 1.1233827769610987
Epoch: 24, Loss: 1.0658535926960304
Epoch: 25, Loss: 1.0188431028979343


In [17]:
model.bias

tensor([-0.1100], dtype=torch.float64, requires_grad=True)

In [18]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.5947983860969543
